# Traveling Wave Detection with MCDA

## Required imports

In [6]:
import h5py
import numpy as np
import pandas as pd
from Detection_functions import MCDA, circular_mask, preprocessing
from moving_cluster_class import Moving_Cluster

## Choose dataset and associated parameters for detection and classification algorithms

In [4]:
condition='awake' #between anesthesia and awake
session='evoked' #between evokedU, evokedD and spontaneous

In [5]:
eps=17
min_pts=32
theta=.40
sign_level=.01
min_dist = 10

## Data loading

In [6]:
original_data_directory=r'/media/mathis/Data/Mathis/Original Data/'

In [7]:
if condition=='awake':
    fs = 100 #Hz
    pixel_spacing = 0.0566 #mm
    if session=='evoked':
        bks_evoked_path = original_data_directory+'gaussian_pos4_ziggy_210908.npy'
        Data = np.load(bks_evoked_path)
    elif session=='spontaneous':
        bks_blank_path = original_data_directory+'blank_ziggy_210908.npy'
        Data = np.load(bks_blank_path)
    mean = np.nanmean(Data)
    nan_loc = np.isnan(Data)
    Data[nan_loc] = mean
    N_trials, Nt, Nx, Ny = Data.shape
    mask = circular_mask((Nx,Ny), center=(220,190), radius=180) #160 should be more relevant

if condition=='anesthesia':
    fs=110 #Hz
    pixel_spacing = 0.0575 #mm
    if session=='evokedD':
        anesth_evokedD_path = original_data_directory+'signal_240306_gaussianD.mat'
        file_anesth_evokedD = h5py.File(anesth_evokedD_path)['signal']
        Data = np.array(file_anesth_evokedD)
    elif session=='evokedU':
        anesth_evokedU_path = original_data_directory+'signal_240306_gaussianU.mat'
        file_anesth_evokedU = h5py.File(anesth_evokedU_path)['signal']
        Data = np.array(file_anesth_evokedU)
    elif session=='spontaneous':
        anesth_blank_path = original_data_directory+'signal_240306_blank.mat' #change it to adapt to your directories
        file_anesth_blank = h5py.File(anesth_blank_path)['signal']
        Data = np.array(file_anesth_blank)
    N_trials, Nt, Nx, Ny = Data.shape
    mask = circular_mask((Nx,Ny), center=(125,140), radius=130) #155 should be more relevant

## Run data-collecting algorithms

In [ ]:
List_dataframes, mc_number = [], 0
for n in range(N_trials):
    preprocessed_trial = preprocessing(trial=Data[n], border_mask=mask, fs=fs, fc=25, threshold=99)
    detected_mc = MCDA(Trial=preprocessed_trial, Epsilon=eps, MinPts=min_pts, Theta=theta, significance_level=sign_level)
    print(f'{len(detected_mc)} moving clusters detected in trial {n}/{N_trials}')
    for mc in detected_mc: 
        moving_cluster=Moving_Cluster(mc['points'], mc['trace'], mc['alpha shapes'], Fs=fs, pixel_spacing=pixel_spacing, condition=condition,session=session, trial=n, number=mc_number)
        moving_cluster.classify(MinDist=min_dist, significance_level=sign_level)
        data = {'MC #':np.full(moving_cluster.length,moving_cluster.number), 'Trial #':np.full(moving_cluster.length, n), 'index':range(moving_cluster.length),
                'Frames': moving_cluster.trace,'Time (ms)': moving_cluster.timepoints, 'Source location':moving_cluster.source,
                'Translation': np.full(moving_cluster.length, moving_cluster.translation),'x_centroid':moving_cluster.centroids[:,0], 'y_centroid':moving_cluster.centroids[:,1], 'Directional speed': moving_cluster.centroid_speed, 
                'Expansion':np.full(moving_cluster.length, moving_cluster.expansion), 'Radial speed': moving_cluster.radial_speed, 'Span': moving_cluster.surfaces}
                #Direction of propagation
                #Latency
                #Crossing
                #V2 area
        df = pd.DataFrame(data)
        List_dataframes.append(df)
        mc_number+=1
Trial_dataframe = pd.concat(List_dataframes, axis=0, ignore_index=True)
print(f'total of {mc_number} detected moving clusters in this dataset')

In [10]:
Trial_dataframe.head(21)

,Wave #,Trial #,index,Frames,Time (ms),x_center,y_center,Center speed (m/s),Velocity,Span (mm2)
0,0,1,0,33,300.000000,221.539604,114.064356,0.112910,-0.011684,14.992191
1,0,1,1,34,309.090909,215.054441,97.432665,0.141367,0.042462,27.729519
2,0,1,2,35,318.181818,206.187234,117.948936,0.147338,0.044040,19.391156
3,0,1,3,36,327.272727,208.756757,141.101351,0.102112,1.839305,11.869438
4,0,1,4,37,336.363636,209.769231,124.989011,0.052197,NaN,7.420878
5,0,1,5,38,345.454545,214.932039,131.427184,NaN,NaN,7.969716
6,1,1,0,40,363.636364,213.662602,136.479675,0.120634,-0.000142,20.173084
7,1,1,1,41,372.727273,213.114815,155.544444,0.022880,-0.016924,21.117019
8,1,1,2,42,381.818182,210.750988,152.806324,0.159244,0.016925,19.035734
9,1,1,3,43,390.909091,210.786802,127.629442,0.166476,-1.579159,16.179134


## Data Frame saving

In [11]:
filename=f'{condition.capitalize()}_{session}_bks.xlsx'
print(filename)

Anesthesia_evokedD_bks.xlsx


In [12]:
#Trial_dataframe.to_excel(filename)